In [140]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [141]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [142]:
demo = pd.read_csv("../data/PTDEMOG.csv")

In [143]:
neuro = pd.read_csv("../data/NEUROEXM.csv")

In [144]:
neuro.columns

Index(['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [145]:
clinical = pd.read_csv("../data/ADSP_PHC_COGN.csv") #.rename(columns={"PHASE":"PHASE"})

In [146]:
clinical.head()

# demo.head()

,RID,SUBJECT_KEY,PHASE,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [147]:
diag["Subject"].value_counts()

Subject
027_S_0120    4
022_S_0096    4
003_S_0981    4
006_S_0498    4
123_S_0072    4
             ..
116_S_0890    1
009_S_1199    1
007_S_1248    1
100_S_1062    1
016_S_6836    1
Name: count, Length: 2875, dtype: int64

In [148]:
comb = pd.read_csv("../data/DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "PHASE"]]

In [149]:
m = comb.merge(demo, on = ["RID", "PHASE"]).merge(neuro,on = ["RID", "PHASE"]).merge(clinical,on = ["RID", "PHASE"]).drop_duplicates()

In [150]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [151]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [152]:
m = m.merge(diag, on = ["PTID", "PHASE"])

In [153]:
m["PTID"].value_counts()

PTID
067_S_4767    18
141_S_0697    15
131_S_0384    15
094_S_1417    14
126_S_1077    14
              ..
068_S_5146     1
033_S_4691     1
041_S_5173     1
010_S_5163     1
131_S_6153     1
Name: count, Length: 2402, dtype: int64

In [154]:
t = m

In [155]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE","DX"], axis=1) 

In [156]:
t.columns

Index(['RID', 'PTID', 'PHASE', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT',
       'PTRTYR', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG',
       'PTADDX', 'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTBORN', 'PTBIRPL', 'PTIMMAGE',
       'PTIMMWHY', 'PTBIRPR', 'PTBIRGR', 'DD_CRF_VERSION_LABEL',
       'LANGUAGE_CODE', 'HAS_QC_ERROR', 'NXVISUAL', 'NXAUDITO'

In [157]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [158]:
len(t.columns)
print(t.columns)
print(cols_to_delete)

Index(['RID', 'PTID', 'PHASE', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME',
       'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR',
       'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT',
       'NXOTHER', 'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')
Index(['PTWORKHS', 'PTWORK', 'PTADBEG', 'PTIDENT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLA

In [159]:
# t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                # "bookkeeper").str.replace("cpa", "accounting")

In [160]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [161]:
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [162]:
# cond = t['PTWORK'].value_counts()
# threshold = 10
# t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [163]:
categorical = ['PTGENDER',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [164]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [165]:
text = ["PTWORK", "CMMED", "PTDOB", "VISDATE"]

In [166]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "PHASE", "RID", "PTID"]))
t[cols_left]

,PTADDX,NXSENSOR,PTETHCAT,PTTLANG,NXABNORM,PHC_MEM,AGE,PHC_EXF,NXMOTOR,EXAMDATE,...,NXGAIT,SUBJECT_KEY,PTRTYR,NXTENDON,NXFINGER,PTRACCAT,NXNERVE,NXPLANTA,NXHEEL,NXOTHER
0,-4.0,1.0,2.0,1.0,1.0,-0.973,82.3655,-1.036,1.0,2006-09-12,...,1.0,G-ADNI-AN000439,1989.0,1.0,1.0,5,1.0,1.0,1.0,1.0
1,-4.0,1.0,2.0,1.0,1.0,-1.068,81.3662,-0.670,1.0,2005-09-12,...,1.0,G-ADNI-AN000439,1989.0,1.0,1.0,5,1.0,1.0,1.0,1.0
2,-4.0,1.0,2.0,1.0,1.0,-1.559,83.3648,-0.645,1.0,2007-09-12,...,1.0,G-ADNI-AN000439,1989.0,1.0,1.0,5,1.0,1.0,1.0,1.0
3,-4.0,1.0,2.0,1.0,1.0,-0.768,81.8645,-0.145,1.0,2006-03-13,...,1.0,G-ADNI-AN000439,1989.0,1.0,1.0,5,1.0,1.0,1.0,1.0
4,-4.0,2.0,2.0,1.0,1.0,0.465,75.7673,0.527,1.0,2007-09-07,...,1.0,G-ADNI-AN000663,1987.0,1.0,1.0,5,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,9999.0,1.0,2.0,1.0,1.0,0.624,80.9801,-4.000,1.0,2019-04-24,...,1.0,-4,2009.0,1.0,1.0,5,1.0,1.0,1.0,1.0
7893,9999.0,1.0,2.0,1.0,1.0,-0.180,76.5996,-4.000,1.0,2020-02-05,...,2.0,-4,2009.0,2.0,1.0,5,1.0,1.0,1.0,-4.0
7894,-4.0,-4.0,2.0,1.0,2.0,0.870,74.7598,-4.000,-4.0,2019-12-04,...,-4.0,-4,2019.0,-4.0,-4.0,5,-4.0,-4.0,-4.0,-4.0
7895,2019.0,1.0,2.0,1.0,1.0,-0.814,74.6804,-4.000,1.0,2020-02-05,...,1.0,-4,2007.0,1.0,2.0,5,1.0,1.0,1.0,2.0


In [167]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXSENSOR
PTETHCAT
PTTLANG
NXABNORM
NXMOTOR
NXCONSCI
NXGAIT
NXTENDON
NXFINGER
PTRACCAT
NXNERVE
NXPLANTA
NXHEEL
NXOTHER


In [168]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY"]
t = t.drop(to_del, axis=1)

In [169]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "PHASE", "RID", "PTID"]))
t[quant]

,PTADDX,PHC_LAN,PHC_VSP,PTCOGBEG,PTDOBYY,PHC_MEM,AGE,PHC_EXF
0,-4.0,-0.828,0.128,-4.0,1924.0,-0.973,82.3655,-1.036
1,-4.0,-0.516,-0.237,-4.0,1924.0,-1.068,81.3662,-0.670
2,-4.0,-0.756,-4.000,-4.0,1924.0,-1.559,83.3648,-0.645
3,-4.0,-0.516,-0.305,-4.0,1924.0,-0.768,81.8645,-0.145
4,-4.0,0.034,0.258,-4.0,1931.0,0.465,75.7673,0.527
...,...,...,...,...,...,...,...,...
7892,9999.0,-4.000,-4.000,2017.0,1938.0,0.624,80.9801,-4.000
7893,9999.0,-4.000,-4.000,2009.0,1943.0,-0.180,76.5996,-4.000
7894,-4.0,-4.000,-4.000,2014.0,1945.0,0.870,74.7598,-4.000
7895,2019.0,-4.000,-4.000,2019.0,1945.0,-0.814,74.6804,-4.000


In [170]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [171]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'PHASE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [172]:
t[l]

,RID,PTID,Group,PHASE,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.973,-1.036,-0.828,0.128
1,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.068,-0.670,-0.516,-0.237
2,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.559,-0.645,-0.756,-4.000
3,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.768,-0.145,-0.516,-0.305
4,5,011_S_0005,0.0,ADNI1,1.0,1931.0,1.0,1.0,16.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,0.465,0.527,0.034,0.258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,6725,013_S_6725,1.0,ADNI3,1.0,1938.0,1.0,3.0,20.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.624,-4.000,-4.000,-4.000
7893,6851,168_S_6851,1.0,ADNI3,2.0,1943.0,1.0,3.0,18.0,1.0,...,1.0,1.0,2.0,1.0,2.0,1.0,-0.180,-4.000,-4.000,-4.000
7894,6841,035_S_6841,0.0,ADNI3,2.0,1945.0,1.0,1.0,14.0,1.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,2.0,0.870,-4.000,-4.000,-4.000
7895,6855,032_S_6855,2.0,ADNI3,1.0,1945.0,1.0,1.0,16.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.814,-4.000,-4.000,-4.000


In [173]:
dfs = []

In [174]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col, dtype=float))

In [175]:
cat = pd.concat(dfs, axis=1)

In [176]:
t[quant]

,PTADDX,PHC_LAN,PHC_VSP,PTCOGBEG,PTDOBYY,PHC_MEM,AGE,PHC_EXF
0,-4.0,-0.828,0.128,-4.0,1924.0,-0.973,82.3655,-1.036
1,-4.0,-0.516,-0.237,-4.0,1924.0,-1.068,81.3662,-0.670
2,-4.0,-0.756,-4.000,-4.0,1924.0,-1.559,83.3648,-0.645
3,-4.0,-0.516,-0.305,-4.0,1924.0,-0.768,81.8645,-0.145
4,-4.0,0.034,0.258,-4.0,1931.0,0.465,75.7673,0.527
...,...,...,...,...,...,...,...,...
7892,9999.0,-4.000,-4.000,2017.0,1938.0,0.624,80.9801,-4.000
7893,9999.0,-4.000,-4.000,2009.0,1943.0,-0.180,76.5996,-4.000
7894,-4.0,-4.000,-4.000,2014.0,1945.0,0.870,74.7598,-4.000
7895,2019.0,-4.000,-4.000,2019.0,1945.0,-0.814,74.6804,-4.000


In [177]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,...,NXPLANTA_-4.0,NXPLANTA_1.0,NXPLANTA_2.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0,NXOTHER_-4.0,NXOTHER_-1.0,NXOTHER_1.0,NXOTHER_2.0
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7893,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7894,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7895,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [178]:
t[["PTID","RID", "PHASE", "Group"]]

,PTID,RID,PHASE,Group
0,011_S_0003,3,ADNI1,2.0
1,011_S_0003,3,ADNI1,2.0
2,011_S_0003,3,ADNI1,2.0
3,011_S_0003,3,ADNI1,2.0
4,011_S_0005,5,ADNI1,0.0
...,...,...,...,...
7892,013_S_6725,6725,ADNI3,1.0
7893,168_S_6851,6851,ADNI3,1.0
7894,035_S_6841,6841,ADNI3,0.0
7895,032_S_6855,6855,ADNI3,2.0


In [179]:
c = pd.concat([t[["PTID", "RID", "PHASE", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [180]:
c

,PTID,RID,PHASE,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,...,NXOTHER_1.0,NXOTHER_2.0,PTADDX,PHC_LAN,PHC_VSP,PTCOGBEG,PTDOBYY,PHC_MEM,AGE,PHC_EXF
0,011_S_0003,3,ADNI1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-4.0,-0.828,0.128,-4.0,1924.0,-0.973,82.3655,-1.036
1,011_S_0003,3,ADNI1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-4.0,-0.516,-0.237,-4.0,1924.0,-1.068,81.3662,-0.670
2,011_S_0003,3,ADNI1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-4.0,-0.756,-4.000,-4.0,1924.0,-1.559,83.3648,-0.645
3,011_S_0003,3,ADNI1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,-4.0,-0.516,-0.305,-4.0,1924.0,-0.768,81.8645,-0.145
4,011_S_0005,5,ADNI1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-4.0,0.034,0.258,-4.0,1931.0,0.465,75.7673,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,013_S_6725,6725,ADNI3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,9999.0,-4.000,-4.000,2017.0,1938.0,0.624,80.9801,-4.000
7893,168_S_6851,6851,ADNI3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,9999.0,-4.000,-4.000,2009.0,1943.0,-0.180,76.5996,-4.000
7894,035_S_6841,6841,ADNI3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,-4.0,-4.000,-4.000,2014.0,1945.0,0.870,74.7598,-4.000
7895,032_S_6855,6855,ADNI3,2.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,2019.0,-4.000,-4.000,2019.0,1945.0,-0.814,74.6804,-4.000


In [181]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [182]:
c.to_csv("clinical.csv")

In [183]:
#reading in the overlap test set
# ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

# #removing ids from the overlap test set
# c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [184]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "PHASE", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_train[:1])

[[ 0.0000000e+00 -4.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00
   1.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  2.6400000e-01  1.0000000e+00
   1.9300000e+03  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  1.0000000e+00  1.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  1.0000000e+00  1.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00 -4.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   1.000

In [187]:
import pickle

print("X_train shape: ", X_train.shape, "y_train shape: ", y_train.shape, "X_test shape: ", X_test.shape, "y_test shape: ", y_test.shape)

with open('X_train_c.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('X_test_c.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open('y_train_c.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open('y_test_c.pkl', 'wb') as f:
    pickle.dump(y_test, f)

# X_train.to_pickle("X_train_c.pkl")
# y_train.to_pickle("y_train_c.pkl")

# X_test.to_pickle("X_test_c.pkl")
# y_test.to_pickle("y_test_c.pkl")

X_train shape:  (2161, 113) y_train shape:  (2161,) X_test shape:  (241, 113) y_test shape:  (241,)
